In [1]:
## If you run for first time uncomment these
# !pip install elasticsearch
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import json
from scipy import spatial
from elasticsearch import Elasticsearch

In [18]:
#Parameters
no_of_recepies = 500 # Total = 124473
ingredient_list = 'Filterd_ingredient' # ingredients (otherwise)

Reading Pickel File for Filtered Data and creating dictionary of recipes

In [35]:
%%time
# Gagan's Code
import pandas as pd
rec_df = pd.read_pickle("Cleaned_recipes.P.zip")
rec_l = []
temp_dict = rec_df.to_dict(orient='index')
for ind in temp_dict:
    rec_l.append(temp_dict[ind])
es = Elasticsearch()
for i in range(0,len(rec_l[:no_of_recepies])): #i took only first 120 recipes because time and size
    res = es.index(index='recipes', doc_type='recipe', id=i, body=rec_l[i])

ValueError: Multiple files found in ZIP file. Only one file per ZIP: ['Cleaned_recipes.P', '__MACOSX/._Cleaned_recipes.P']

In [3]:
# %%time
# recipes = []
# with open('recipes_raw_nosource_ar.json') as json_file: #make sure the recipe json is in same dir
#     data = json.load(json_file)
#     for p in data:
#         recipes.append(data[p])
# es = Elasticsearch()
# for i in range(0,len(recipes[:500])): #i took only first 120 recipes because time and size
#     res = es.index(index='recipes', doc_type='recipe', id=i, body=recipes[i])

Wall time: 1min 26s


In [25]:
def giveResults(query, last, wish,title=True,instructions=True):
    res = es.search(index="recipes", body={"query": {"bool": { "must": [{"match":{ingredient_list:query} }]}},"size":no_of_recepies})
    mscores = [] # new scores for personalized results
    m2scores = [] #new scores for advanced personalized results
    results = [] #unpersonalized
    presults = [] #personalized 
    p2results = [] #adv personalized
    for hit in res['hits']['hits']:
        if title:
            results.append((hit['_source']['title'],hit['_score']))
        if instructions: 
            results.append((hit['_source']['instructions'],hit['_score']))
        ingr = ''
        for s in hit['_source'][ingredient_list]:
            ingr += s
        ldocuments = (ingr,last)
        wdocuments = (ingr,wish)
        tfidf_vectorizer=TfidfVectorizer()
        tfidf_last=tfidf_vectorizer.fit_transform(ldocuments)
        tfidf_wish=tfidf_vectorizer.fit_transform(wdocuments)
        cs = -100 * cosine_similarity(tfidf_last[0:1],tfidf_last)
        ds = 100 * cosine_similarity(tfidf_wish[0:1],tfidf_wish)
        mscores.append((hit['_source'],hit['_score']+cs[0][1]))
        m2scores.append((hit['_source'],hit['_score']+cs[0][1]+ds[0][1]))
    mscores.sort(key=lambda t: t[1],reverse=True)
    m2scores.sort(key=lambda t: t[1],reverse=True)
    for t in mscores:
        if title:
            presults.append((t[0]['title'],t[1]))
        if instructions:
            presults.append((t[0]['instructions'],t[1]))
    for t in m2scores:
        if title:
            p2results.append((t[0]['title'],t[1]))
        if instructions:
            p2results.append((t[0]['instructions'],t[1]))
    return presults,p2results, results

In [26]:
def getInput():
    query = input("Enter whatever ingredients you have in your pantry seperated by whitespaces: ")
    last = input("Enter at least 5 ingredients you ate last week seperated by whitespace: ")
    wish = input("List anything you would like to eat. You can also just skip this.")
    return query,last,wish

In [29]:
def giveRecipes(title = True, instructions = False):
    query,last,wish = getInput()
    print(query)
    personalized, personalized2, standard = giveResults(query,last,wish,title,instructions)
    return personalized, personalized2, standard

In [30]:
p1,p2,s = giveRecipes()

Enter whatever ingredients you have in your pantry seperated by whitespaces: tofu tuna beef
Enter at least 5 ingredients you ate last week seperated by whitespace: fish cheese rice
List anything you would like to eat. You can also just skip this.meat
tofu tuna beef


In [31]:
s[:10] #standard

[('Easy Tuna Casserole', 5.712923),
 ('Best Tuna Casserole', 4.882046),
 ("Barbie's Tuna Salad", 4.450528),
 ('Tuna Noodle Casserole from Scratch', 4.0890985),
 ('Cabbage Roll Casserole', 3.4308121),
 ('High Temperature Eye-of-Round Roast', 3.1109436),
 ("Shepherd's Pie VI", 2.9607046),
 ('BBQ Pork for Sandwiches', 2.9238453),
 ('Bacon Wrapped Smokies', 2.9238453),
 ('Slow Cooker Beef Stew I', 2.873215)]

In [33]:
p1[:10] #personalized using only last week

[('Easy Tuna Casserole', 5.712923),
 ('Best Tuna Casserole', 4.882046),
 ("Barbie's Tuna Salad", 4.450528),
 ('Cabbage Roll Casserole', 3.4308121),
 ('High Temperature Eye-of-Round Roast', 3.1109436),
 ('BBQ Pork for Sandwiches', 2.9238453),
 ('Bacon Wrapped Smokies', 2.9238453),
 ('Slow Cooker Beef Stew I', 2.873215),
 ('Foolproof Rib Roast', 2.8384893),
 ('Hamburger Steak with Onions and Gravy', 2.8313808)]

In [34]:
p2[:10]#personalized using last week and wish

[('Slow Cooker Beef Stew I', 22.233539687858617),
 ('Slow Cooker Beef Stroganoff I', 16.907940277983457),
 ('Easy Tuna Casserole', 5.712923),
 ('Best Tuna Casserole', 4.882046),
 ("Barbie's Tuna Salad", 4.450528),
 ('Cabbage Roll Casserole', 3.4308121),
 ('High Temperature Eye-of-Round Roast', 3.1109436),
 ('BBQ Pork for Sandwiches', 2.9238453),
 ('Bacon Wrapped Smokies', 2.9238453),
 ('Foolproof Rib Roast', 2.8384893)]